In [10]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import required libararies

In [11]:
import mlflow

import pandas as pd

# Load the evaluation dataset

In [12]:
import ast


def parse_workup(workup_str: str) -> dict:
    """Parse JSON strings into actual dictionaries."""
    return ast.literal_eval(workup_str)


def to_evaluation_format(row: pd.Series) -> dict:
    """Convert the DataFrame into the required evaluation format."""
    return {
        "inputs": {"scenario": row["Rephrased"]},
        "expectations": {
            "rating": row["Synthetic_life_rating"],
            "workup": parse_workup(row["Synthetic_life_rating_workup"]),
        },
        "tags": {"id": row["DeID"]},
    }


data = pd.read_csv("../data/rating_data.csv")


# Define systems to compare

In [13]:
from typing import Any, Protocol


class PredictionSystem(Protocol):
    def predict(self, scenario: str) -> Any: ...

In [ ]:
from pydantic import BaseModel, Field
from workflow2reasoning.core.agents.underwriting import UnderwritingAgent


class ConditionRating(BaseModel):
    name: str
    condition_rating: str


class WorkflowUWResponse(BaseModel):
    rating: str | None = None
    conditions: list[ConditionRating] = Field(default_factory=list)
    error: str | None = None


class WorkflowSystem(PredictionSystem):
    def __init__(self, model_name: str):
        self.agent = UnderwritingAgent().compile()
        self.model_name = model_name

    @mlflow.trace
    def predict(self, scenario: str) -> WorkflowUWResponse:
        mlflow.update_current_trace(tags={"id": id})
        try:
            response = self.agent.invoke(
                {"medical_summary": scenario},
                config={
                    "max_concurrency": 4,
                    "recursion_limit": 300,
                    "configurable": {
                        "model_name": self.model_name,
                    },
                },
            )
            response = WorkflowUWResponse(
                rating=response["overall_rating"]["life_rating"],
                conditions=[
                    ConditionRating(
                        name=impairment,
                        condition_rating=trace["rating"]["co-morbidity"],
                    )
                    for impairment, trace in response["trace"].items()
                ],
            )
        except Exception as e:
            response = WorkflowUWResponse(error=str(e))
        return response

In [15]:
from uw_baseline.agent import get_underwriter, UnderwritingResponse
from langchain.chat_models import init_chat_model
from langchain_core.rate_limiters import InMemoryRateLimiter


class ReActResponse(UnderwritingResponse):
    rating: str | None = None
    error: str | None = None


class ReActSystem(PredictionSystem):
    def __init__(self, model_name: str):
        rate_limiter = None
        if rate_settings := self.get_rate_limit_settings(model_name):
            rate_limiter = InMemoryRateLimiter(
                requests_per_second=rate_settings["requests_per_second"],
                check_every_n_seconds=0.1,
                max_bucket_size=rate_settings["max_bucket_size"],
            )
        self.model = init_chat_model(model_name, rate_limiter=rate_limiter)
        self.agent = get_underwriter(model=self.model)

    @mlflow.trace
    def predict(self, scenario: str) -> UnderwritingResponse | dict:
        try:
            response = self.agent.invoke(
                {"messages": [("user", scenario)]},
                {"config": {"recursion_limit": 50, "retry_limit": 10}},
            )
        except Exception as e:
            return ReActResponse(rating="", conditions=[], error=str(e))
        return response["structured_response"]

    def get_rate_limit_settings(self, model_name: str) -> dict:
        settings = {
            "azure_openai:gpt-4o": {"requests_per_second": 40, "max_bucket_size": 42},
            "azure_openai:gpt-4o-mini": {
                "requests_per_second": 18,
                "max_bucket_size": 19,
            },
            "azure_openai:gpt-4.1": {"requests_per_second": 2, "max_bucket_size": 2},
            "azure_openai:gpt-4.1-mini": {
                "requests_per_second": 46,
                "max_bucket_size": 48,
            },
            "azure_openai:gpt-4.1-nano": {
                "requests_per_second": 41,
                "max_bucket_size": 44,
            },
            "azure_openai:gpt-5": {"requests_per_second": 80, "max_bucket_size": 84},
            "azure_openai:gpt-5-mini": {"requests_per_second": 3, "max_bucket_size": 4},
            "azure_openai:o1-mini": {"requests_per_second": 3, "max_bucket_size": 4},
        }
        return settings.get(
            model_name.replace("azure_openai:", ""), None
        )  # Default to empty dict if model not found

# Define Experiment

In [16]:
import mlflow
import pandas as pd
from evals.scorers import rating_is_exact_match, condition_details_match


def run_evaluations(
    data: pd.DataFrame,
    change_id: str,
    change_description: str,
    models: list,
    systems: dict[str, PredictionSystem],
):
    for model_name in models:
        for system_name, system_cls in systems.items():
            # System to evaluate
            system = system_cls(model_name=model_name)
            # Run Name
            provider, model = model_name.split(":")
            run_name = f"{system_name}-{model}-{change_id}"
            with mlflow.start_run(
                run_name=run_name,
                description=f"Evaluating {system_name} with {model} for the following changes: {change_description}",
                nested=True,
            ) as run:
                mlflow.log_table(data, "ground_truth.json")
                mlflow.log_params(
                    {
                        "change_id": change_id,
                        "change_description": change_description,
                        "system": system_name,
                        "model": model,
                        "provider": provider,
                        "num_samples": len(data),
                    }
                )
                mlflow.genai.evaluate(
                    data=data.apply(to_evaluation_format, axis=1).to_list(),
                    predict_fn=system.predict,
                    scorers=[rating_is_exact_match, condition_details_match],
                )
                print(f"Completed run: {run.info.run_id} - {run_name}")
    print("All evaluations completed.")

# Run the experiments

In [ ]:
change_id = "v18"  # could be a git sha or other unique identifier
change_description = "Running all available hosted open source and OpenAI models on reAct system."  # What did you change
n = data.shape[0]  # data.shape[0] for all data
systems = {
    "reAct": ReActSystem,
    "workflow": WorkflowSystem,
}
models = [
    ## Azure OpenAI Models
    "azure_openai:gpt-4o",
    "azure_openai:gpt-4o-mini",
    "azure_openai:gpt-4.1",
    "azure_openai:gpt-4.1-mini",
    "azure_openai:gpt-4.1-nano",
    "azure_openai:gpt-5",
    "azure_openai:gpt-5-mini",
    ## Open Source Models hosted on Databricks (REQUIRES OPENAI_ENDPOINT AND OPENAI_API_KEY to be set in env)
    # "openai:databricks-claude-3-7-sonnet",
    # "openai:databricks-claude-sonnet-4",
    # "openai:databricks-llama-4-maverick",  # Does not support jsonSchema additionalProperties... Use extra='forbid' in pydantic model to disable additionalProperties.
    # "openai:databricks-meta-llama-3-1-8b-instruct",  # Does not support jsonSchema additionalProperties... Use extra='forbid' in pydantic model to disable additionalProperties., Takes too long to think, need to increase recursion limit...
    # "openai:databricks-meta-llama-3-3-70b-instruct",  # Does not support jsonSchema additionalProperties... Use extra='forbid' in pydantic model to disable additionalProperties.
    ### Below are incompatible models and their specific issues:
    # "openai:databricks-meta-llama-3-1-405b-instruct",  # unsupported additionalProperties on json schema for tool calling i.e JsonSchema v1 only!, no token limit available.
    # "openai:databricks-gpt-oss-120b",  # No tool use and response format use together.
    # "openai:databricks-gpt-oss-20b",  # No tool use and response format use together.
    # "openai:databricks-gemma-3-12b",  # unsupported additionalProperties on json schema for tool calling i.e JsonSchema v1 only. No multi-turn tool calls...
    # "azure_openai:o1-mini" # No tool use.
]
run_evaluations(data[:n], change_id, change_description, models, systems)

2025/10/10 16:43:31 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/10/10 16:43:31 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.


Evaluating:   0%|          | 0/104 [Elapsed: 00:00, Remaining: ?] 

Evaluation completed.

Metrics and evaluation results can be viewed from the MLflow run page.
To compare evaluation results across runs, view the "Evaluations" tab of the experiment.

Get aggregate metrics: `result.metrics`.
Get per-row evaluation results: `result.tables['eval_results']`.
`result` is the `EvaluationResult` object returned by `mlflow.evaluate`.

Completed run: 79dcff17075945488f974194998af3a4 - reAct-gpt-4o-v18
🏃 View run reAct-gpt-4o-v18 at: https://adb-2498687920111167.7.azuredatabricks.net/ml/experiments/2187665894323267/runs/79dcff17075945488f974194998af3a4
🧪 View experiment at: https://adb-2498687920111167.7.azuredatabricks.net/ml/experiments/2187665894323267


2025/10/10 16:47:04 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
2025/10/10 16:47:04 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.


INFO     | 2025-10-10 16:47:04.141 | workflow2reasoning.core.agents.underwriting:underwriting_manager:94 - Calling profiling agent to get patient profiling information
INFO     | 2025-10-10 16:47:05.182 | workflow2reasoning.core.agents.underwriting:underwriting_manager:98 - Calling impairment agent to get patient impairments
INFO     | 2025-10-10 16:47:05.191 | workflow2reasoning.core.agents.riskfactorupdatedoptimized:extract_impairments_node:68 - [extract_impairments_node] Starting extraction of impairments from medical summary.
INFO     | 2025-10-10 16:47:15.617 | workflow2reasoning.core.agents.riskfactorupdatedoptimized:extract_impairments_node:90 - Output from extract_impairments_node: {
  "impairment_relevant_snippets": [
    "height 6 feet, weight 260 lbs",
    "type 2 diabetes diagnosed at age 55 and is currently taking Metformin 500 mg, Glipizide 5 mg, and Ozempic 0.25 mg for diabetes management",
    "His most recent A1C is 7.4 (last tested 8/2024)",
    "hypertension, treated

...

INFO     | 2025-10-10 16:54:21.837 | workflow2reasoning.core.agents.comorbidityoptimized:execute_all_edges:106 - Executing node: execute_all_edges
INFO     | 2025-10-10 16:54:21.883 | workflow2reasoning.core.agents.singlegraphoptimized:execute_next_step:184 - Output: step_reasoning='This step focuses on assessing diabetes-specific complications and applying additional debits accordingly. Based on the medical summary provided, the applicant does not exhibit any of the complications listed in the underwriting manual, such as microalbuminuria, retinopathy, neuropathy, amputation, renal impairment, or ulcers. His diabetes is well controlled, with an A1C level of 5.8 after treatment initiation and lifestyle changes, and no complications have been reported.' step_output='No debits applied' is_relevant=True
INFO     | 2025-10-10 16:54:22.290 | workflow2reasoning.core.agents.comorbidityoptimized:get_edges:87 - Executing node: get_edges
INFO     | 2025-10-10 16:54:22.522 | workflow2reasoning.co

/Users/aggaani/projects/work/opsman2plan/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:290: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


INFO     | 2025-10-10 16:54:22.756 | workflow2reasoning.core.agents.comorbidityoptimized:execute_all_edges:106 - Executing node: execute_all_edges
INFO     | 2025-10-10 16:54:22.860 | workflow2reasoning.core.agents.singlegraphoptimized:execute_next_step:152 - Executing node: execute_next_step
INFO     | 2025-10-10 16:54:22.871 | workflow2reasoning.core.agents.singlegraphoptimized:execute_next_step:160 - Current step index: 4
INFO     | 2025-10-10 16:54:25.075 | workflow2reasoning.core.agents.underwriting:underwriting_manager:98 - Calling impairment agent to get patient impairments
INFO     | 2025-10-10 16:54:25.362 | workflow2reasoning.core.agents.riskfactorupdatedoptimized:extract_impairments_node:68 - [extract_impairments_node] Starting extraction of impairments from medical summary.
INFO     | 2025-10-10 16:54:25.817 | workflow2reasoning.core.agents.singlegraphoptimized:execute_next_step:184 - Output: step_reasoning="The applicant is a 57-year-old male with Type 2 diabetes diagnosed

/Users/aggaani/projects/work/opsman2plan/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:290: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


INFO     | 2025-10-10 16:54:33.073 | workflow2reasoning.core.agents.processgraph:block_for_batch_completion:80 - Impairments remaining:  set()
INFO     | 2025-10-10 16:54:33.076 | workflow2reasoning.core.agents.processgraph:block_for_batch_completion:87 - Trace: {'Diabetes Mellitus': {'life_rating': '+0', 'ltc': '+0', 'wp': '+0', 'adb': '+0'}, 'Build-Risk Assessment': {'life_rating': '+0', 'ltc': '+0', 'wp': '+0', 'adb': '+0'}}
INFO     | 2025-10-10 16:54:33.076 | workflow2reasoning.core.agents.processgraph:block_for_batch_completion:87 - Trace: {'Diabetes Mellitus': {'life_rating': '+0', 'ltc': '+0', 'wp': '+0', 'adb': '+0'}, 'Build-Risk Assessment': {'life_rating': '+0', 'ltc': '+0', 'wp': '+0', 'adb': '+0'}}
